In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import glob

# Classify using reservoir causality data

In [2]:
# load computed causality matrices
con_files = [f for f in glob.glob("data/sub-CON*.npy")]
pat_files = [f for f in glob.glob("data/sub-PAT*.npy")]

# filter out excluded subjects
excluded = ["PAT08", "CON03", "PAT03", "PAT11"]
con_files = [f for f in con_files if f[9:14] not in excluded]
pat_files = [f for f in pat_files if f[9:14] not in excluded]

# transform for LDA
con_causalities = np.array([np.load(f) for f in con_files])
pat_causalities = np.array([np.load(f) for f in pat_files])
con_flattened = con_causalities.reshape((con_causalities.shape[0], -1))
pat_flattened = pat_causalities.reshape((pat_causalities.shape[0], -1))

In [37]:
#merge the con and pat matrixes into a single dataset
X = np.concatenate((con_flattened, pat_flattened), axis=1)
X[np.isnan(X)] = 0
labels_con = [1]*5
labels_pat = [-1]*5
y = np.array(labels_con + labels_pat)

In [5]:
from sklearn.model_selection import ShuffleSplit

In [14]:
rs = ShuffleSplit(n_splits=4, test_size=.3, random_state=0)
rs.get_n_splits(X, y, ['control', 'patient'])

4

In [46]:
from sklearn import svm

for train_index, test_index in rs.split(X, y):
    clf = svm.SVC(kernel = 'rbf')
    clf.fit(X[train_index], y[train_index])
    y_pred = clf.predict(X[test_index])
    s = svm.SVC.score(clf, X[test_index], y[test_index])
    print(s)

0.3333333333333333
0.3333333333333333
0.3333333333333333
0.3333333333333333
